In [1]:
# Importowanie Bibliotek
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import kerastuner as kt

C:\Users\Micha\AppData\Local\Temp\ipykernel_29016\1514695378.py:15: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [2]:
# Przygotowanie Danych

# Wczytanie danych z pliku CSV
data = pd.read_csv('../data.csv')

# Wybór tylko numerycznych kolumn do treningu
numeric_columns = ['Size(L)', 'OG', 'FG', 'IBU', 'Color', 'BoilSize', 'BoilTime', 'BoilGravity', 'Efficiency']
X = data[numeric_columns].copy()

# Uzupełnienie brakujących wartości średnią
X = X.fillna(X.mean())

# Przygotowanie etykiet (zawartości alkoholu)
y = data['ABV'].copy()

# Normalizacja danych
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [3]:
# Utworzenie Modelu

# Definiowanie modelu sekwencyjnego z warstwami Dense
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),  # Warstwa wejściowa
    BatchNormalization(),  # Normalizacja wsadowa
    Dropout(0.5),  # Dropout dla regularizacji
    Dense(128, activation='relu'),  # Warstwa ukryta
    BatchNormalization(),  # Normalizacja wsadowa
    Dropout(0.5),  # Dropout dla regularizacji
    Dense(64, activation='relu'),  # Warstwa ukryta
    BatchNormalization(),  # Normalizacja wsadowa
    Dropout(0.5),  # Dropout dla regularizacji
    Dense(32, activation='relu'),  # Warstwa ukryta
    BatchNormalization(),  # Normalizacja wsadowa
    Dropout(0.5),  # Dropout dla regularizacji
    Dense(1)  # Warstwa wyjściowa do regresji
])

# Kompilacja modelu
model.compile(optimizer=Adam(learning_rate=0.001),  # Użycie optymalizatora Adam z określoną szybkością uczenia
              loss='mean_squared_error',  # Funkcja straty do regresji
              metrics=['mean_absolute_error'])  # Metryka do oceny modelu

# Wyświetlenie podsumowania modelu
model.summary()  # Podsumowanie architektury modelu

c:\Users\Micha\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │         2,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,745 (186.50 KB)

 Trainable params: 46,785 (182.75 KB)

 Non-trainable params: 960 (3.75 KB)

In [4]:
import kerastuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

# Listy możliwych wartości dla hiperparametrów
units_layer_1_list = [128, 160, 192, 224, 256, 288, 320, 352, 384, 416, 448, 480, 512]
units_layer_2_list = [64, 96, 128, 160, 192, 224, 256]
units_layer_3_list = [32, 48, 64, 80, 96, 112, 128]
units_layer_4_list = [16, 32, 48, 64]
dropout_list = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
learning_rate_list = [1e-4, 3e-4, 1e-3, 3e-3, 1e-2]
batch_size_list = [16, 32, 48, 64, 80, 96, 112, 128]

# Funkcja do budowania modelu
def build_model(hp):
    model = Sequential()
    
    # Warstwa wejściowa
    model.add(Dense(
        units=hp.Choice('units_layer_1', values=units_layer_1_list),  # Wybór z listy
        activation='relu',
        input_shape=(X_train.shape[1],)
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_1', values=dropout_list)))  # Wybór z listy
    
    # Warstwa ukryta 1
    model.add(Dense(
        units=hp.Choice('units_layer_2', values=units_layer_2_list),  # Wybór z listy
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_2', values=dropout_list)))  # Wybór z listy
    
    # Warstwa ukryta 2
    model.add(Dense(
        units=hp.Choice('units_layer_3', values=units_layer_3_list),  # Wybór z listy
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_3', values=dropout_list)))  # Wybór z listy
    
    # Warstwa ukryta 3
    model.add(Dense(
        units=hp.Choice('units_layer_4', values=units_layer_4_list),  # Wybór z listy
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_4', values=dropout_list)))  # Wybór z listy
    
    # Warstwa wyjściowa
    model.add(Dense(1))  # Warstwa wyjściowa do regresji
    
    # Kompilacja modelu
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=learning_rate_list)),  # Wybór z listy
        loss='mean_squared_error',
        metrics=['mean_absolute_error']
    )
    
    return model

In [5]:
# Definiowanie tunera
tuner = kt.BayesianOptimization(build_model,
    objective='val_mean_absolute_error',
    max_trials=10,
    directory='../models',
    project_name='regresja_alkoholu'
)
# Dodanie callbacków do wczesnego zatrzymania i redukcji współczynnika uczenia
early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=5, 
                               restore_best_weights=True)

# Dodanie callbacka do TensorBoard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='../models/regresja_alkoholu')

# Przeszukiwanie hiperparametrów
tuner.search(X_train, y_train, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[early_stopping, tensorboard_callback],
             batch_size=32)

Trial 10 Complete [00h 00m 39s]
val_mean_absolute_error: 0.3440142869949341

Best val_mean_absolute_error So Far: 0.22376391291618347
Total elapsed time: 00h 06m 19s


In [6]:
# Pobranie najlepszego modelu
best_model = tuner.get_best_models(num_models=1)[0]

# Wyświetlenie podsumowania najlepszego modelu
best_model.summary()

c:\Users\Micha\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Micha\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 320)            │         3,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 320)            │         1,280 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 320)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 48)             │         1,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 48)             │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            49 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,313 (114.50 KB)

 Trainable params: 28,385 (110.88 KB)

 Non-trainable params: 928 (3.62 KB)

In [8]:
# Trening najlepszego modelu
history = best_model.fit(
    X_train, y_train,
    epochs=20,
    validation_split=0.2,
    batch_size=32,
    callbacks=[early_stopping, tensorboard_callback],
)

# Print model summary
best_model.summary()

final_val_mae = history.history['val_mean_absolute_error'][-1]
print(f"Final Validation MAE: {final_val_mae:.2f}")

Epoch 1/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.1184 - mean_absolute_error: 0.5531 - val_loss: 0.3851 - val_mean_absolute_error: 0.3531
Epoch 2/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.3353 - mean_absolute_error: 0.5640 - val_loss: 0.2703 - val_mean_absolute_error: 0.2523
Epoch 3/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.0568 - mean_absolute_error: 0.5463 - val_loss: 0.5865 - val_mean_absolute_error: 0.5547
Epoch 4/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.2050 - mean_absolute_error: 0.5621 - val_loss: 0.4903 - val_mean_absolute_error: 0.4258
Epoch 5/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.1570 - mean_absolute_error: 0.5454 - val_loss: 0.3730 - val_mean_absolute_error: 0.3373
Epoch 6/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.9099 - mean_absolute_error: 0.5396 - val_loss: 0.4058 - val_mean_absolute_error: 0.3475
Epoch 7/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.1140 - mean_absolute_err

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 320)            │         3,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 320)            │         1,280 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 320)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 48)             │         1,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 48)             │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            49 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 86,085 (336.27 KB)

 Trainable params: 28,385 (110.88 KB)

 Non-trainable params: 928 (3.62 KB)

 Optimizer params: 56,772 (221.77 KB)

Final Validation MAE: 0.28


In [10]:
%load_ext tensorboard
%tensorboard --logdir ../models/regresja_alkoholu

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 21400), started 0:05:00 ago. (Use '!kill 21400' to kill it.)